# Google Forms API - Onboarding Questionnaire

In [1]:
#%pip install google-api-python-client
#%pip install gspread

### Libraries

In [2]:
import json
import vertexai
from google.cloud import storage
from googleapiclient.discovery import build
from google.oauth2 import service_account
import os
from dotenv import load_dotenv

### Google Forms

In [3]:
# Load Credentials From A Service Account JSON File
credentials = service_account.Credentials.from_service_account_file('/home/jupyter/3. Onboarding Experience/serviceaccount.json')

In [4]:
# Define API Version and Service Name
api_version = 'v1'
service_name = 'forms'

In [5]:
# Service Object for Interacting with the Google Forms API
service = build(service_name, api_version, credentials=credentials)

In [6]:
form = {
    "info": {
        "title": "Profile Creation with CogniEdu"
    },
}

In [7]:
NEW_QUESTION = {
    "requests": [
        {
            "createItem": {
                "item": {
                    "title": "Question 1: How intense would you like your study plan to be, considering your daily commitments and learning goals?",
                    "questionItem": {
                        "question": {
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Casual (I am not on a strict time-crunch, I want to slowly get into learning the material)"},
                                    {"value": "Standard (I have assignments to complete and material to learn, I have things deadlines I want to meet)"},
                                    {"value": "High Impact (I am on a time-crunch and want to follow the most efficient studying strategies I can)"}
                                ]
                            },
                            "required": True
                        }
                    }
                },
                "location": {"index": 0}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 2: What is your preferred studying time during WEEKDAYS?",
                    "questionItem": {
                        "question": {
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Morning (~6am-11am)"},
                                    {"value": "Afternoons (11am-5pm)"},
                                    {"value": "Night (6pm-10pm)"},
                                    {"value": "Midnight (10pm-3am)"},
                                    {"value": "No studying on the weekdays"}
                                ]
                            },
                            "required": True
                        }
                    }
                },
                "location": {"index": 1}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 3: What is your preferred studying time during WEEKENDS?",
                    "questionItem": {
                        "question": {
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Morning (~6am-11am)"},
                                    {"value": "Afternoons (11am-5pm)"},
                                    {"value": "Night (6pm-10pm)"},
                                    {"value": "Midnight (10pm-3am)"},
                                    {"value": "No studying on the weekends"}
                                ]
                            },
                            "required": True
                        }
                    }
                },
                "location": {"index": 2}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 4: How would you prefer your study sessions?",
                    "questionItem": {
                        "question": {
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Segmented (ideal for students with several breaks between scheduled appointments)"},
                                    {"value": "Chunks (ideal for students with a packed schedule and then large chunks of free time)"}
                                ]
                            },
                            "required": True
                        }
                    }
                },
                "location": {"index": 3}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 5: What are the challenges you face when studying? (Why do you need CogniBot’s assistance?)",
                    "questionItem": {
                        "question": {
                            "textQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 4}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 6: What motivates you to study?",
                    "questionItem": {
                        "question": {
                            "choiceQuestion": {
                                "type": "RADIO",
                                "options": [
                                    {"value": "Higher Education"},
                                    {"value": "Personal Growth"},
                                    {"value": "Social Competition"},
                                    {"value": "Requirement"}
                                ]
                            },
                            "required": True
                        }
                    }
                },
                "location": {"index": 5}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 7: When do you start your day on the weekdays?",
                    "questionItem": {
                        "question": {
                            "timeQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 6}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 8: When do you end your day on the weekdays?",
                    "questionItem": {
                        "question": {
                            "timeQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 7}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 9: When do you start your day on the weekends?",
                    "questionItem": {
                        "question": {
                            "timeQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 8}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 10: When do you end your day on the weekends?",
                    "questionItem": {
                        "question": {
                            "timeQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 9}
            }
        },
        {
            "createItem": {
                "item": {
                    "title": "Question 11: Please list the title of the courses you are taking this semester/quarter",
                    "questionItem": {
                        "question": {
                            "textQuestion": {},
                            "required": True
                        }
                    }
                },
                "location": {"index": 10}
            }
        }
    ]
}

In [8]:
result = service.forms().create(body=form).execute()

In [9]:
question_setting = (
    service.forms()
    .batchUpdate(formId=result["formId"], body=NEW_QUESTION)
    .execute()
)

In [10]:
get_result = service.forms().get(formId=result["formId"]).execute()
#print(get_result)

In [11]:
# Form URL Generated From Response
form_url = get_result['responderUri']
#print('Form URL:', form_url)

### Google Forms Responses

In [12]:
response = service.forms().responses().list(formId=result["formId"]).execute()
#print(response)

### Create JSON Files for Each User & Load into GCS Bucket

In [13]:
load_dotenv()

PROJECT_ID = os.getenv('PROJECT_ID')
LOCATION = os.getenv('LOCATION')

vertexai.init(project=PROJECT_ID, location=LOCATION)

BUCKET = os.getenv('BUCKET')

In [14]:
client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(BUCKET)

In [15]:
responses = response.get('responses', [])

In [16]:
for response_data in responses:
    response_id = response_data['responseId']
    
    # File Name for JSON File
    filename = f"response_{response_id}.json"
    
    # Write Response Data to JSON File
    with open(filename, 'w') as json_file:
        json.dump(response_data, json_file)

    print(f"Success: JSON File Created\n")
    
    # Path to JSON File in Bucket
    blob = bucket.blob(filename)
    
    # Upload JSON file to Bucket
    blob.upload_from_filename(filename)
    
    print(f"Success: JSON File Uploaded to GCS Bucket")